In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
from xgboost.sklearn import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import classification_report,roc_auc_score,accuracy_score

### load data

In [28]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,id,MO HLADR+ MFI (cells/ul),Neu CD64+MFI (cells/ul),CD3+T (cells/ul),CD8+T (cells/ul),CD4+T (cells/ul),NK (cells/ul),CD19+ (cells/ul),CD45+ (cells/ul),Age,Sex 0M1F,Mono CD64+MFI (cells/ul),label
0,0,3556.0,2489.0,265.19,77.53,176.55,0.00,4.20,307.91,52,0,7515.0,1
1,1,1906.0,134.0,1442.61,551.90,876.07,112.10,168.15,1735.48,20,1,1756.0,0
2,2,1586.0,71.0,1332.74,684.20,655.26,244.95,216.52,1820.04,28,1,1311.0,0
3,3,683.0,94.0,419.23,255.80,162.17,72.05,44.68,538.22,55,1,1443.0,0
4,4,1032.0,71.0,1102.72,480.27,625.30,188.78,130.77,1427.97,28,1,1542.0,0


In [29]:
train.shape

(87, 13)

In [30]:
train[train.isnull().values==True]

,id,MO HLADR+ MFI (cells/ul),Neu CD64+MFI (cells/ul),CD3+T (cells/ul),CD8+T (cells/ul),CD4+T (cells/ul),NK (cells/ul),CD19+ (cells/ul),CD45+ (cells/ul),Age,Sex 0M1F,Mono CD64+MFI (cells/ul),label
39,39,NaN,NaN,1336.54,739.71,550.3,68.46,192.07,1615.68,21,0,NaN,0
39,39,NaN,NaN,1336.54,739.71,550.3,68.46,192.07,1615.68,21,0,NaN,0
39,39,NaN,NaN,1336.54,739.71,550.3,68.46,192.07,1615.68,21,0,NaN,0


In [31]:
train.dropna(axis=0, how='any', inplace=True)
train.shape

(86, 13)

In [32]:
sm = SMOTE(random_state = 500)
train,y = sm.fit_resample(train[[i for i in train.columns if i not in['id','label']]], train['label'])
train['label'] = y
print(train.shape,train['label'].sum())

(114, 12) 57


In [33]:
scaler = MinMaxScaler()
columns = ['MO HLADR+ MFI (cells/ul)', 'Neu CD64+MFI (cells/ul)',
       'CD3+T (cells/ul)', 'CD8+T (cells/ul)', 'CD4+T (cells/ul)',
       'NK (cells/ul)', 'CD19+ (cells/ul)', 'CD45+ (cells/ul)','Age', 'Mono CD64+MFI (cells/ul)']
scaler.fit(train)

scaler.fit(train[columns])

train[columns] = scaler.transform(train[columns])
test[columns] = scaler.transform(test[columns])

In [34]:
X_train,y_train = train[[i for i in train.columns if i not in['id','label']]],train['label']
                        
X_test = test[[i for i in train.columns if i not in['id','label']]]

### Random Forest

In [43]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(150,min_samples_split = 5)
rf.fit(X_train,y_train)

RandomForestClassifier(min_samples_split=5, n_estimators=150)

In [45]:
threshold = 0.5

predicted_proba = rf.predict_proba(X_train)

y_pred = (predicted_proba [:,1] >= threshold).astype('int')

print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        57
           1       1.00      0.98      0.99        57

    accuracy                           0.99       114
   macro avg       0.99      0.99      0.99       114
weighted avg       0.99      0.99      0.99       114



In [46]:
predicted_proba = rf.predict_proba(X_test)

prediction = (predicted_proba [:,1] >= threshold).astype('int')

pd.DataFrame({"id":range(59),"label":prediction}).to_csv("submission.csv",index=None)

### Reference: 
https://github.com/shawinbetter/MSBD5001-Spring-2022-Individual-Project/blob/main/code.ipynb